# Getting Started

该笔记本介绍了如何开始使用聊天模型。


In [1]:
import os
import sys               
sys.path.append(os.path.dirname(os.path.abspath(".."))) 
from API_KEYS import InitAzureEnv,AzureDeploymentName

InitAzureEnv()

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
chat = AzureChatOpenAI(
    deployment_name=AzureDeploymentName.CHAT,
    temperature=0
)

您可以通过将一条或多条消息传递给聊天模型来获得聊天完成。响应将是一条消息。`LangChain` 目前支持的消息类型有`AIMessage`, `HumanMessage`, `SystemMessage`, 和`ChatMessage–ChatMessage`接受任意角色参数。大多数时候，你只会处理`HumanMessage`, `AIMessage`, 和`SystemMessage`


In [4]:
chat([HumanMessage(content="把这段话从中文翻译为英文:我喜欢编程")])

AIMessage(content='I like programming.', additional_kwargs={}, example=False)

OpenAI 的聊天模型支持多条消息作为输入。有关更多信息，请参见[此处](https://platform.openai.com/docs/guides/chat/chat-vs-completions)。以下是向聊天模型发送系统和用户消息的示例：


In [6]:
messages = [
    SystemMessage(content="你是一个翻译助理，如果对方说的是中文，你要他说的话翻译为英文"),
    HumanMessage(content="我喜欢编程.")
]
chat(messages)

AIMessage(content='I like programming.', additional_kwargs={}, example=False)

您可以更进一步，使用`generate`为多组消息生成补全信息。这将返回 LLMResult 带有附加 message 参数的 。


In [7]:
batch_messages = [
    [
        SystemMessage(content="你是一个翻译助理，如果对方说的是中文，你要他说的话翻译为英文."),
        HumanMessage(content="我喜欢编程.")
    ],
    [
        SystemMessage(content="你是一个翻译助理，如果对方说的是中文，你要他说的话翻译为英文."),
        HumanMessage(content="我喜欢看电影.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='I like programming.', generation_info=None, message=AIMessage(content='I like programming.', additional_kwargs={}, example=False))], [ChatGeneration(text='I like watching movies.', generation_info=None, message=AIMessage(content='I like watching movies.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 115, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo'})

您可以从此`LLMResult`中查看`token`使用之类的内容


In [8]:
result.llm_output

{'token_usage': {'completion_tokens': 9,
  'prompt_tokens': 115,
  'total_tokens': 124},
 'model_name': 'gpt-3.5-turbo'}

## PromptTemplates


您可以通过使用`MessagePromptTemplate`从一个或多个构建一个`MessagePromptTemplates`。您可以使用`ChatPromptTemplate`的`format_prompt`返回一个`PromptValue`，您可以将其转换为字符串或`Message`对象，具体取决于您是要使用格式化值作为`llm`还是聊天模型的输入。


In [10]:
system_msg_prompt = SystemMessagePromptTemplate.from_template("你是是一个翻译助手，请将 {input_language} 翻译为 {output_language}.")
human_msg_prompt = HumanMessagePromptTemplate.from_template("{text}")

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])
chat(chat_prompt.format_prompt(input_language="中文", output_language="英文", text="我喜欢编程").to_messages())

AIMessage(content='I like programming.', additional_kwargs={}, example=False)

如果想更直接的构造 MessagePromptTemplate，可以在外面创建一个 PromptTemplate，然后传入，


In [11]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

## LLMChain

您可以使用与之前非常相似的方式使用现有的`LLMChain`提供一个提示和模型


In [12]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [13]:
chain.run(input_language="正文", output_language="英文", text="我喜欢编程.")

'I like programming.'

## Streaming

Streaming is supported for `ChatOpenAI` through callback handling.


In [14]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = AzureChatOpenAI(
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()], 
    temperature=0,
    deployment_name=AzureDeploymentName.CHAT
)
resp = chat([HumanMessage(content="香港有哪些著名的电影演员?")])


1. 周润发
2. 梁朝伟
3. 张国荣
4. 刘德华
5. 王家卫
6. 林青霞
7. 张曼玉
8. 汤姆·克鲁斯
9. 吴彦祖
10. 古天乐
11. 陈奕迅
12. 周星驰
13. 李连杰
14. 成龙
15. 姜文
16. 刘青云
17. 谢霆锋
18. 郑伊健
19. 余文乐
20. 黄秋生

## 提供一些镜头示例

### 交替的人类/人工智能信息

进行少量镜头提示的第一种方法依赖于使用交替的人/人工智能信息。请参阅下面的示例。


In [19]:
template="你是一个翻译助理，如果对方说的是中文，你要他说的话翻译为英文."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = SystemMessagePromptTemplate.from_template("你好", additional_kwargs={"name": "example_human"})
example_ai = SystemMessagePromptTemplate.from_template("Hi", additional_kwargs={"name": "example_ai"})
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [20]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
# get a chat completion from the formatted messages
chain.run("我喜欢编程.")

I enjoy programming.

'I enjoy programming.'